<a href="https://colab.research.google.com/github/Nuthan03/NLP/blob/main/Introduction_to_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Intorduction to NLP Fundamentals in TensorFlow

NLP has the goal of deriving information out of natural language(could be seqeuences of texts or speech).
Another common term for NLP problems is sequence to sequence problems(seq2seq)

In [1]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-2dddbe78-a459-5f11-9302-e9032f3cc194)


## Get the helper functions


In [2]:
# Download helper functions script
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

# Import series of helper function for the notebook
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

--2023-12-14 07:07:51--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2023-12-14 07:07:51 (69.8 MB/s) - ‘helper_functions.py’ saved [10246/10246]



## Get a text dataset
The dataset we are going to be using us Kaggle's introduction to NLP datasets (text samples of Tweets labelled as diaster or not diaster).

In [3]:
# Download data (same as from Kaggle)
!wget "https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip"

# Unzip data
unzip_data("nlp_getting_started.zip")

--2023-12-14 07:07:56--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.197.207, 74.125.135.207, 74.125.142.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.197.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.01s   

2023-12-14 07:07:56 (60.3 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



## Visualizing a text dataset

To visualize our text samples, we first have to read them in, one way to do so would be to use Python: https://realpython.com/read-write-files-python/


In [4]:
import pandas as pd
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [5]:
# Shuffle training dataframe
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [6]:
# What does the test dataframe look like ?
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [7]:
# How many examples of each class?
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [8]:
# How many total samples?
len(train_df), len(test_df)

(7613, 3263)

In [9]:
# Let's visualize some random training examples
import random
random_index = random.randint(0, len(train_df)-5)
for row in train_df_shuffled[["text", "target"]][random_index:random_index+5].itertuples():
  _, text, target = row
  print(f"Target: {target}", "(real diaster)" if target > 0 else "(not real diaster)")
  print(f"Text:\n{text}\n")
  print("---\n")

Target: 0 (not real diaster)
Text:
#hot  Funtenna: hijacking computers to send data as sound waves [Black Hat 2015] http://t.co/aAtt5aMnmD #prebreak #best

---

Target: 1 (real diaster)
Text:
UNPREDICTABLE DISCONNECTED AND SOCIAL CASUALTY ARE MY FAVORITES HOW DO PEOPLE NOT LIKE THEM

---

Target: 1 (real diaster)
Text:
For these Californians it's like living in Africa's Sahel desert - just finding water a daily chore. http://t.co/ySG9vsrT4g

---

Target: 1 (real diaster)
Text:
Myanmar Flooding: Heavy monsoon rains during the month of July have caused flooding flash floods and landsli... http://t.co/9TG7A5OqFP

---

Target: 0 (not real diaster)
Text:
I've just watched episode S01E09 of Bloody Monday! http://t.co/vRptHvPymt #bloodymonday #tvshowtime http://t.co/NkKvknBvOz

---



## Split data into training and validation sets

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
# Use train_test_split training data into training and validation sets
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                            train_df_shuffled["target"].to_numpy(),
                                                                            test_size=0.1, # use 10% of training data for validation
                                                                            random_state=42)

In [12]:
# Check the lengths
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

In [13]:
# View the first 10 training sentences and their labels
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object),
 array([0,

## Converting text into numbers

When dealing with a text problem, one of the first things you'll have to do before you can build a model is to convert your text to numbers.

There are a few ways to do this namely:
* Tokenziation-direct mapping of token(a token could be a word or a character) to number
* Embedding-create a martrix of feature vector for each token(the size of the feature vector can be defined and this embedding can be learned)

### Text vectorization(tokenization)

In [14]:
train_sentences[:5]

array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
       "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
       'Somehow find you and I collide http://t.co/Ee8RpOahPk'],
      dtype=object)

In [15]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization

# Use the default TextVectorization variables
text_vectorizer = TextVectorization(max_tokens=None, # How many words in the vocabullary
                                    standardize="lower_and_strip_punctuation", # How to process text
                                    split="whitespace", # How to split tokens
                                    ngrams=None, # Grouping words is none
                                    output_mode="int", # how to map tokens to numbers
                                    output_sequence_length=None) # How long should the output sequence of tokens be?
                                    #pad_to_max_tokens=True) # Not valid if using max_tokens=None

In [16]:
# Find the average number of tokens (words) in the training tweets
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [17]:
# Setup text vectorization wtth custom variables
max_vocab_length = 10000 # max number of words to have in our vocabulary
max_length = 15

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode="int",
                                    output_sequence_length=max_length)

In [18]:
# Fit the text vectorizer to the training text
text_vectorizer.adapt(train_sentences)

In [19]:
# Create a sample sentence and tokenize it
sample_sentence = "There's a flood in my street!"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [20]:
# Choose a random sentence from the training dataset and tokenize it
random_sentence = random.choice(train_sentences)
print(f"Original text:\n {random_sentence}\
        \n\nVectorized version:")
text_vectorizer([random_sentence])

Original text:
 @TheHammers_ @tonycottee1986 alsowhat if some of the 1st team players got injured?Then Bilic would get slated for playing themhe can't win        

Vectorized version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[7650, 7483,    1,   47,   82,    6,    2, 1337,  697, 2797,  100,
           1,    1,   78,   52]])>

In [21]:
# Get the unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5]
bottom_5_words = words_in_vocab[-5:]
print(f"Number of words in vocab: {len(words_in_vocab)}")
print(f"Top 5 most common words: {top_5_words}")
print(f"Bottom 5 least common words: {bottom_5_words}")

Number of words in vocab: 10000
Top 5 most common words: ['', '[UNK]', 'the', 'a', 'in']
Bottom 5 least common words: ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


### Creating an Embedding using an Embedding Layer

To make our embedding, we are going to use TensorFlow's embedding layer: https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding

The parameters we care most about for our embedding layer:


*   `input_dim` = the size of our vocabulary.
*   `output_dim` = the size of the output embedding vector, for example, a value of 100 would mean each token gets represented by a vector 100 long
*   `input_length` = length of the sequences being passed to the embedding layer



In [22]:
tf.random.set_seed(42)
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim=max_vocab_length, # set input shape
                             output_dim=128, # set size of embedding vector
                             embeddings_initializer="uniform",
                             input_length=max_length, # how long is each input
                             name="embedding")

embedding


In [23]:
# Get a random sentence from training set
random_sentence = random.choice(train_sentences)
print(f"Original text:\n{random_sentence}\
       \n\nEmbedded version")

# Embed the random sentence (turn it into numerical representation)
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text:
@Erker Again?? Eep! Thought of you yesterday when I saw that hella scary hail. #armageddon?       

Embedded version


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.00790269,  0.04789729,  0.03157226, ..., -0.04954776,
         -0.04415081, -0.02399202],
        [ 0.00152786, -0.01075239, -0.04324482, ..., -0.0133575 ,
          0.02062323,  0.03713143],
        [ 0.00790269,  0.04789729,  0.03157226, ..., -0.04954776,
         -0.04415081, -0.02399202],
        ...,
        [-0.00715532, -0.020576  , -0.03858091, ...,  0.00905042,
          0.02606924,  0.01339242],
        [ 0.01643548,  0.02296526, -0.02350134, ...,  0.0145157 ,
          0.03873901,  0.01004878],
        [-0.00895153,  0.04865017,  0.0316245 , ...,  0.00200826,
          0.04793617, -0.02755984]]], dtype=float32)>

In [24]:
# Chcek out a single token's embedding
sample_embed[0][0], sample_embed[0][0].shape, random_sentence[0]

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([ 0.00790269,  0.04789729,  0.03157226, -0.02445362,  0.02698337,
         0.0342429 ,  0.03227549, -0.02362095, -0.01287609,  0.01577767,
        -0.0037305 ,  0.00830092,  0.03981281, -0.02846769, -0.02905035,
        -0.00045963, -0.04116126, -0.01681847, -0.04364539,  0.00169792,
         0.04766715, -0.00067735,  0.00826182, -0.03606018,  0.00531193,
         0.03351508,  0.03480956, -0.04215158,  0.00945539, -0.02387127,
        -0.03801798, -0.02027348,  0.02720639, -0.04098973,  0.01858696,
        -0.03056531, -0.03494084, -0.00700815, -0.03730339,  0.04265955,
         0.00638604, -0.0030866 , -0.03565508, -0.03486491, -0.04653521,
         0.03145443, -0.02771716,  0.04465696,  0.03964363,  0.01955818,
        -0.04764303,  0.03516296, -0.0071316 , -0.03543977,  0.00207018,
         0.03769685, -0.00711217,  0.02184523, -0.0114452 , -0.02278712,
         0.01654411,  0.03155888, -0.04950975,  0.03792298, -0.04406143,
  

## Modelling a text dataset

Now we have got way to turn our text sequences into numbers, it's time to start building a series of modelling experiments.

We'll start with a baseline and move on from there.

* Model 0: Navie Bayes(baseline), this is from Sklearn ML map.
* Model 1: Feed-forward neural network(dense model)
* Model 2: LSTM model(RNN)
* Model 3: GRU model(RNN)
* Model 4: Bidirectional-LSTM model(RNN)
* Model 5: 1D Convolutional Neural Network(CNN)
* Model 6: TensorFlow Hub Pretrained Feature Extractor(using transfer learning for NLP)
* Model 7: Same as model 6 with 10% of training data

How are we going to apporach all these 6 model?

Use the standard steps in modelling with tensorflow:

* Create a model
* Build a model
* Fit a model
* Evaluate our model

### Model 0: Getting a baseline

As with all machine learning modelling experiments, it's important to create a baseline model so you've got a benchmark for future experimentsto build upon.

To create our baseline, we'll use Sklearn's Multinomial Naive Bayes using the TF-IDF formula to convert our words to numbers.

>`⛏Note:` It's common pratice to use non-DL algorithms as a baseline because of their speed and then later using DL to see if you can improve upon them.

In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Create tokenization and modelling pipeline
model_0 = Pipeline([
    ("tfidf", TfidfVectorizer()), # Convert words to numebr using tfidf
    ("clif", MultinomialNB()) # model the text
])

# Fit the pipeline to the training data
model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clif', MultinomialNB())])

In [26]:
# Evaluate the baseline model
baseline_score = model_0.score(val_sentences, val_labels)
print(f"Our baseline model achives an accuracy of: {baseline_score*100:.2f}%")

Our baseline model achives an accuracy of: 79.27%


In [27]:
# Make predictions
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

### Creating an evaluation function for our model experiments

we could evaluate these as they are but since we are going to be evaluating several models in the same way going forward, let's create a helper function which takes an array of predictions and ground truth labels and computes the following:

 * Accuracy
 * Precision
 * Recall
 * F1-score
> `⚓Note:`Since we are dealing with a classfication problem, the above metrics are the most appropriate. If we are working with a regression problem, other metrics such as MAE (mean absolute error) would be a better choice


In [28]:
# Function to evaluate: accuracy, precision, recall, f1-score
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculaute_results(y_true, y_pred):
  """
  Calculates model accuracy, precision, recall and f1-score of a binary classification model.
  """
  # Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  # Calculate model precision , recall and f1 score using "weighted" average
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {"accuracy": model_accuracy,
                   "precision": model_precision,
                   "recall": model_recall,
                   "f1": model_f1}
  return model_results

In [30]:
# Get baseline results
baseline_results = calculaute_results(y_true=val_labels,
                                      y_pred=baseline_preds)
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

### Model 1: A simple dense model

#### Creating `wandb callback` since Tendorboard will be shutdown from 2024


In [33]:
!pip install wandb

In [34]:
import wandb
from wandb.keras import WandbCallback
import datetime

def create_wandb_callback(dir_name, experiment_name):
  """
  Create a Wandb callback to store experiment logs.
  """
  wandb.init(project=dir_name, name=experiment_name)
  print(f"Saving Wandb logs to directory: {dir_name}, experiment: {experiment_name}")

  return WandbCallback()

In [35]:
# Create a dir to save Wandb logs
SAVE_DIR = "model_logs"

In [36]:
# Build model with the Functional API
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_1 = tf.keras.Model(inputs, outputs, name="model_1_dense")

In [38]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (  (None, 128)               0         
 GlobalAveragePooling1D)                                         
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1280129 (4.88 MB)
Trainable params: 128

In [39]:
# Compile model_1
model_1.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [45]:
model_1_history = model_1.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks = [create_wandb_callback(SAVE_DIR,
                                                                 "simple_dense_model")])

accuracy,▁
epoch,▁
loss,▁
val_accuracy,▁
val_loss,▁
accuracy,0.82134
best_epoch,0
best_val_loss,0.47245
epoch,0
loss,0.43972
val_accuracy,0.78346


Saving Wandb logs to directory: model_logs, experiment: simple_dense_model
Epoch 1/5
212/215 [============================>.] - ETA: 0s - loss: 0.3457 - accuracy: 0.8604

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
wandb: ERROR Can't save model in the h5py format. The model will be saved as as an W&B Artifact in the 'tf' format.
wandb: Adding directory to artifact (/content/wandb/run-20231214_084200-4j735sab/files/model-best)... Done. 0.1s


215/215 [==============================] - 2s 11ms/step - loss: 0.3457 - accuracy: 0.8603 - val_loss: 0.4614 - val_accuracy: 0.7940
Epoch 2/5
215/215 [==============================] - 2s 10ms/step - loss: 0.2834 - accuracy: 0.8905 - val_loss: 0.4648 - val_accuracy: 0.7795
Epoch 3/5
215/215 [==============================] - 2s 8ms/step - loss: 0.2359 - accuracy: 0.9133 - val_loss: 0.4803 - val_accuracy: 0.7795
Epoch 4/5
215/215 [==============================] - 1s 6ms/step - loss: 0.1996 - accuracy: 0.9289 - val_loss: 0.5018 - val_accuracy: 0.7822
Epoch 5/5
215/215 [==============================] - 1s 7ms/step - loss: 0.1708 - accuracy: 0.9397 - val_loss: 0.5248 - val_accuracy: 0.7822


In [46]:
# Check the results
model_1.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 5ms/step - loss: 0.5248 - accuracy: 0.7822


[0.5247884392738342, 0.7821522355079651]

In [48]:
# Make some predictions and evalaute those
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs.shape

24/24 [==============================] - 0s 5ms/step


(762, 1)

In [51]:
model_1_pred_probs[:10]

array([[0.4304058 ],
       [0.7404586 ],
       [0.99921715],
       [0.08010492],
       [0.03646898],
       [0.96222234],
       [0.90269953],
       [0.99810684],
       [0.99187875],
       [0.3422793 ]], dtype=float32)

In [53]:
# Covert model prediction probabilities to label format
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))
model_1_preds[:20]

<tf.Tensor: shape=(20,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1.], dtype=float32)>

In [54]:
# Calclate our model_1 results
model_1_results = calculaute_results(y_true=val_labels,
                                     y_pred=model_1_preds)
model_1_results

{'accuracy': 78.21522309711287,
 'precision': 0.7849731976577976,
 'recall': 0.7821522309711286,
 'f1': 0.7797935460636591}

In [55]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

In [56]:
import numpy as np
np.array(list(model_1_results.values())) > np.array(list(baseline_results.values()))

array([False, False, False, False])

##Visualizing learned embeddings

In [57]:
# Get the vocabulary from the text vectorization layer
words_in_vocab = text_vectorizer.get_vocabulary()
len(words_in_vocab), words_in_vocab[:10]

(10000, ['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is'])

In [60]:
# Model 1 summary
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (  (None, 128)               0         
 GlobalAveragePooling1D)                                         
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1280129 (4.88 MB)
Trainable params: 128

In [62]:
# Get the weight matrix of embedding layer
# (these are the numerical patterns between the text in the training dataset the model has learned)
embed_weights = model_1.get_layer("embedding").get_weights()[0]
print(embed_weights.shape)

(10000, 128)


In [63]:
embed_weights

array([[-0.0083241 , -0.05826063,  0.02003638, ..., -0.00294417,
        -0.03549365, -0.04941987],
       [ 0.00520919,  0.04518729,  0.03375654, ..., -0.04719815,
        -0.046947  , -0.02673827],
       [-0.02214554, -0.05422305,  0.01543618, ...,  0.05785286,
        -0.04323579, -0.02656462],
       ...,
       [ 0.0143656 ,  0.01572814, -0.02324056, ..., -0.00072576,
        -0.04013902, -0.04385969],
       [-0.07927047, -0.00416491,  0.05947633, ...,  0.03557833,
        -0.01390036, -0.01831763],
       [-0.06882343, -0.06185576,  0.05291457, ...,  0.12412279,
        -0.06489011, -0.09364115]], dtype=float32)

Now we have got the embedding matrix our model has learned to represent our tokens, let's see how we can visualize our embedding.

To use the Embedding vectors (same as embedding weights).

* The embedding vectors (same as embedding weights).
* The meta data of the embedding vectors(the words they represent-our vocabulary).


In [ ]:
# Create embedding files (we got this TensorFlow's word embeddings documentation)
